In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('/kaggle/input/lish-moa/train_features.csv')
data.head()

In [ ]:
target = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')

In [ ]:
# target

In [ ]:
D = target[target.columns[1:]]

In [ ]:
cols = list(target.columns[1:])
labelss = []
for j in tqdm(range(len(target))):
    for i in range(len(cols)):
        if D.loc[j][cols[i]] != 0:
            labelss.append(cols[i])
            break
        if i == len(cols)-1:
                labelss.append('NULL')

In [ ]:
from catboost import Pool, CatBoostClassifier

train_data = pd.read_csv('/kaggle/input/lish-moa/train_features.csv')[data.columns[4:]]

eval_data = pd.read_csv('/kaggle/input/lish-moa/test_features.csv')[data.columns[4:]]

# cat_features = [0]

train_label = labelss
# eval_label = ["USA", "France", "USA", "UK"]


train_dataset = Pool(data=train_data,
                     label=train_label)

eval_dataset = Pool(data=eval_data)

# Initialize CatBoostClassifier
model = CatBoostClassifier(iterations=150,
                           learning_rate=0.7,
                           depth=7,
                           loss_function='MultiClass',
                           task_type = 'GPU')
# Fit model
model.fit(train_dataset)
# Get predicted classes
preds_class = model.predict(eval_dataset)
# Get predicted probabilities for each class
preds_proba = model.predict_proba(eval_dataset)
# Get predicted RawFormulaVal
# preds_raw = model.predict(eval_dataset, 
#                           prediction_type='RawFormulaVal')

In [ ]:
pred = pd.DataFrame(preds_proba,columns=list(set(labelss)))

In [ ]:
pred['sig_id'] = pd.read_csv('/kaggle/input/lish-moa/test_features.csv')['sig_id']

In [ ]:
pred['erbb2_inhibitor']=np.zeros(len(eval_data))
pred['norepinephrine_reuptake_inhibitor']=np.zeros(len(eval_data))

In [ ]:
pred = pred[pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv').columns]

In [ ]:
pred.to_csv('submission.csv',index=None)

In [ ]:
pred